In [2]:
import sys
sys.path.insert(0, '/cndd/fangming/CEMBA/snmcseq_dev')
import importlib

from __init__ import *
from __init__jupyterlab import *
import snmcseq_utils
importlib.reload(snmcseq_utils)
import CEMBA_preproc_utils
importlib.reload(CEMBA_preproc_utils)
import CEMBA_clst_utils

from scipy.io import mmread
from scipy import sparse
import time

import fbpca

8 datasets (modalities)
- Expression values (raw or normalized) in anndata.X
- At least one embedding (e.g., tSNE, UMAP) in anndata.obsm, specified with the prefix X_ (e.g., by default scanpy stores UMAP coordinates in anndata.obsm['X_umap'])
- A unique identifier for every cell is available in an anndata.obs field (you can specify this with the --obs-names option)
- A unique identifier for every gene is available in an anndata.var field (you can specify which field to use with the --var-names option)

In [3]:
data_dir = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons'
sys.path.insert(0, data_dir)
from __init__datasets import *

In [4]:
mods

('snmcseq_gene',
 'snatac_gene',
 'smarter_cells',
 'smarter_nuclei',
 '10x_cells_v2',
 '10x_cells_v3',
 '10x_nuclei_v3',
 '10x_nuclei_v3_macosko')

### Getting metadata  

In [5]:
# coordinates; cluster labels
f = '/cndd/fangming/CEMBA/data/MOp_all/results_final/miniatlas_fig4_scf_clusterings.tsv'
meta = pd.read_csv(f, sep='\t')
print(meta.shape)
meta.head()

(408885, 11)


/cndd/fangming/venvs/conda_dobro/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,sample,joint_embedding_x,joint_embedding_y,modality,modality_name,single_modality_cluster,single_modality_annot,joint_cluster_round1,joint_cluster_round2,joint_cluster_round3,joint_cluster_round4
0,snmcseq_gene_2C_M_0,0.471629,-3.134735,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Tenm2,L4-IT-Rorb_Rorb-Tenm2,1,1-1,1-1-1,1-1-1-1
1,snmcseq_gene_2C_M_1,-2.414973,-12.098081,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,4,4-2,4-2-1,4-2-1-1
2,snmcseq_gene_2C_M_100,-1.446751,-12.051894,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,4,4-1,4-1-1,4-1-1-1
3,snmcseq_gene_2C_M_1000,-1.901734,-13.660591,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,4,4-2,4-2-1,4-2-1-1
4,snmcseq_gene_2C_M_1001,-0.981086,-11.936621,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,4,4-1,4-1-1,4-1-1-1


In [6]:
f = '/cndd/fangming/CEMBA/data/MOp_all/results_final/miniatlas_fig4_scf_annotation_round1.tsv'
annot1 = pd.read_csv(f, sep='\t')
print(annot1.shape)
annot1.head()

f = '/cndd/fangming/CEMBA/data/MOp_all/results_final/miniatlas_fig4_scf_annotation_round2.tsv'
annot2 = pd.read_csv(f, sep='\t')
print(annot2.shape)
annot2.head()

f = '/cndd/fangming/CEMBA/data/MOp_all/results_final/miniatlas_fig4_scf_annotation_round3.tsv'
annot3 = pd.read_csv(f, sep='\t')
print(annot3.shape)
annot3.head()

(10, 3)
(29, 3)
(56, 3)


,cluster_id_iterative,annotation_curated,color
0,1-1-1,L4/5 IT (1),#3CBC78
1,1-2-1,L4/5 IT (2),#3CBC45
2,1-2-2,L4/5 IT (3),#5DDB65
3,2-1-1,L6 CT Cpa6 (1),#338C5E
4,2-1-2,L6 CT Cpa6 (2),#338C5E


In [7]:
tmp = pd.merge(meta, 
             annot1.rename(columns={'annotation_curated': 'joint_cluster_round1_annotation'})
             [['cluster_id_iterative', 'joint_cluster_round1_annotation']], 
             left_on='joint_cluster_round1', right_on='cluster_id_iterative').drop('cluster_id_iterative', axis=1)
tmp.head()

,sample,joint_embedding_x,joint_embedding_y,modality,modality_name,single_modality_cluster,single_modality_annot,joint_cluster_round1,joint_cluster_round2,joint_cluster_round3,joint_cluster_round4,joint_cluster_round1_annotation
0,snmcseq_gene_2C_M_0,0.471629,-3.134735,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Tenm2,L4-IT-Rorb_Rorb-Tenm2,1,1-1,1-1-1,1-1-1-1,L4/5 IT
1,snmcseq_gene_2C_M_1006,4.204999,-2.770652,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Ryr3,L4-IT-Rorb_Rorb-Ryr3,1,1-2,1-2-1,1-2-1-1,L4/5 IT
2,snmcseq_gene_2C_M_1009,9.292183,-5.484662,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Cpne4,L4-IT-Rorb_Rorb-Cpne4,1,1-2,1-2-2,1-2-2-3,L4/5 IT
3,snmcseq_gene_2C_M_101,5.540587,-2.440171,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Cpne4,L4-IT-Rorb_Rorb-Cpne4,1,1-2,1-2-2,1-2-2-6,L4/5 IT
4,snmcseq_gene_2C_M_1011,-0.091199,-2.604921,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Tenm2,L4-IT-Rorb_Rorb-Tenm2,1,1-1,1-1-1,1-1-1-1,L4/5 IT


In [8]:
tmp = pd.merge(tmp, 
             annot2.rename(columns={'annotation_curated': 'joint_cluster_round2_annotation'})
             [['cluster_id_iterative', 'joint_cluster_round2_annotation']], 
             left_on='joint_cluster_round2', right_on='cluster_id_iterative').drop('cluster_id_iterative', axis=1)
tmp.head()

,sample,joint_embedding_x,joint_embedding_y,modality,modality_name,single_modality_cluster,single_modality_annot,joint_cluster_round1,joint_cluster_round2,joint_cluster_round3,joint_cluster_round4,joint_cluster_round1_annotation,joint_cluster_round2_annotation
0,snmcseq_gene_2C_M_0,0.471629,-3.134735,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Tenm2,L4-IT-Rorb_Rorb-Tenm2,1,1-1,1-1-1,1-1-1-1,L4/5 IT,L4/5 IT (1)
1,snmcseq_gene_2C_M_1011,-0.091199,-2.604921,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Tenm2,L4-IT-Rorb_Rorb-Tenm2,1,1-1,1-1-1,1-1-1-1,L4/5 IT,L4/5 IT (1)
2,snmcseq_gene_2C_M_1012,-0.037576,-8.106225,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,1,1-1,1-1-1,1-1-1-1,L4/5 IT,L4/5 IT (1)
3,snmcseq_gene_2C_M_1022,0.906631,-0.785923,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Cpne4,L4-IT-Rorb_Rorb-Cpne4,1,1-1,1-1-1,1-1-1-1,L4/5 IT,L4/5 IT (1)
4,snmcseq_gene_2C_M_1024,1.627603,-1.945875,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Cpne4,L4-IT-Rorb_Rorb-Cpne4,1,1-1,1-1-1,1-1-1-1,L4/5 IT,L4/5 IT (1)


In [9]:
tmp = pd.merge(tmp, 
             annot3.rename(columns={'annotation_curated': 'joint_cluster_round3_annotation'})
             [['cluster_id_iterative', 'joint_cluster_round3_annotation']], 
             left_on='joint_cluster_round3', right_on='cluster_id_iterative').drop('cluster_id_iterative', axis=1)
tmp.head()

,sample,joint_embedding_x,joint_embedding_y,modality,modality_name,single_modality_cluster,single_modality_annot,joint_cluster_round1,joint_cluster_round2,joint_cluster_round3,joint_cluster_round4,joint_cluster_round1_annotation,joint_cluster_round2_annotation,joint_cluster_round3_annotation
0,snmcseq_gene_2C_M_0,0.471629,-3.134735,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Tenm2,L4-IT-Rorb_Rorb-Tenm2,1,1-1,1-1-1,1-1-1-1,L4/5 IT,L4/5 IT (1),L4/5 IT (1)
1,snmcseq_gene_2C_M_1011,-0.091199,-2.604921,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Tenm2,L4-IT-Rorb_Rorb-Tenm2,1,1-1,1-1-1,1-1-1-1,L4/5 IT,L4/5 IT (1),L4/5 IT (1)
2,snmcseq_gene_2C_M_1012,-0.037576,-8.106225,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,1,1-1,1-1-1,1-1-1-1,L4/5 IT,L4/5 IT (1),L4/5 IT (1)
3,snmcseq_gene_2C_M_1022,0.906631,-0.785923,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Cpne4,L4-IT-Rorb_Rorb-Cpne4,1,1-1,1-1-1,1-1-1-1,L4/5 IT,L4/5 IT (1),L4/5 IT (1)
4,snmcseq_gene_2C_M_1024,1.627603,-1.945875,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Cpne4,L4-IT-Rorb_Rorb-Cpne4,1,1-1,1-1-1,1-1-1-1,L4/5 IT,L4/5 IT (1),L4/5 IT (1)


In [21]:
# output = '/cndd2/portal_dev/metadata_for_cellxgene_8datasets.tsv'
# tmp.to_csv(output, sep='\t', header=True, index=False, na_rep='NA')

### Load Yang's data 

In [10]:
f = '/cndd2/fangming/projects/miniatlas/data/Table_S4_UnimodalClusters/Table_S4g_open_chromatin_metadata.tsv.gz'
df_old = pd.read_csv(f, sep='\t')
print(df_old.shape)
f = '/cndd2/fangming/projects/miniatlas/data/MOpAlignToRS1.meta.txt'
df_new = pd.read_csv(f, sep='\t')
print(df_new.shape)

(81196, 13)
(81196, 10)


In [11]:
df_old.head()

,cell,sample,barcode,Num fragments,Num uniquely mapped,Num proper paired,Num unique fragments,Num chrM fragments,cluster,tsne1,tsne2,umap-1,umap-2
0,CEMBA171206_3C_AGCGATAGAACCAGGTAAGAGATGTATAGCCT,CEMBA171206_3C,AGCGATAGAACCAGGTAAGAGATGTATAGCCT,138330,128263,128039,64868,0,L6.CT,7.605117,0.697672,-2.125204,-4.396653
1,CEMBA171206_3C_AGCGATAGAACCAGGTAATGACGTCAGGACGT,CEMBA171206_3C,AGCGATAGAACCAGGTAATGACGTCAGGACGT,159133,146663,146348,69167,0,L23.b,-9.492925,-6.200938,8.959074,3.006819
2,CEMBA171206_3C_AGCGATAGAACCAGGTAGGATAACATAGAGGC,CEMBA171206_3C,AGCGATAGAACCAGGTAGGATAACATAGAGGC,19152,17586,17545,8348,0,ASC,14.227358,-11.023748,-6.880754,-2.903271
3,CEMBA171206_3C_AGCGATAGAACCAGGTATAGCCTTAGGCGAAG,CEMBA171206_3C,AGCGATAGAACCAGGTATAGCCTTAGGCGAAG,14876,13837,13801,6583,0,L6.CT,5.144768,4.790674,-0.955707,-4.635163
4,CEMBA171206_3C_AGCGATAGAACCAGGTATAGCCTTATAGAGGC,CEMBA171206_3C,AGCGATAGAACCAGGTATAGCCTTATAGAGGC,28532,26303,26202,12437,0,L5.IT.b,5.218895,-8.674734,0.796851,4.447933


In [12]:
df_new.head()

,sample,barcode,cluster,Class,MajorType,SubType,ClassColor,MajorTypeColor,SubTypeColor,note
0,CEMBA171206_3C,AGCGATAGAACCAGGTAAGAGATGTATAGCCT,L6.CT,NaN,NaN,NaN,NaN,NaN,NaN,lowQuality
1,CEMBA171206_3C,AGCGATAGAACCAGGTAATGACGTCAGGACGT,L23.b,Glutamate,ITL23GL,ITL23GL1,#8C4E3C,#E59D9F,#9F5B16,used
2,CEMBA171206_3C,AGCGATAGAACCAGGTAGGATAACATAGAGGC,ASC,NonN,ASC,ASCC,#704E89,#A13F9E,#9C5B7E,used
3,CEMBA171206_3C,AGCGATAGAACCAGGTATAGCCTTAGGCGAAG,L6.CT,Glutamate,CTGL,CTGL4,#8C4E3C,#A78533,#EEC4AD,used
4,CEMBA171206_3C,AGCGATAGAACCAGGTATAGCCTTATAGAGGC,L5.IT.b,Glutamate,ITL6GL,ITL6GL1,#8C4E3C,#9E4954,#8C5E19,used


In [15]:
df_new_update = df_new.copy()
df_new_update['cell'] = df_new['sample'] + "_" + df_new['barcode']
df_new_update.columns = "Li et al 2020 " + df_new_update.columns

print(df_new_update.shape)
df_new_update.head()

(81196, 11)


,Li et al 2020 sample,Li et al 2020 barcode,Li et al 2020 cluster,Li et al 2020 Class,Li et al 2020 MajorType,Li et al 2020 SubType,Li et al 2020 ClassColor,Li et al 2020 MajorTypeColor,Li et al 2020 SubTypeColor,Li et al 2020 note,Li et al 2020 cell
0,CEMBA171206_3C,AGCGATAGAACCAGGTAAGAGATGTATAGCCT,L6.CT,NaN,NaN,NaN,NaN,NaN,NaN,lowQuality,CEMBA171206_3C_AGCGATAGAACCAGGTAAGAGATGTATAGCCT
1,CEMBA171206_3C,AGCGATAGAACCAGGTAATGACGTCAGGACGT,L23.b,Glutamate,ITL23GL,ITL23GL1,#8C4E3C,#E59D9F,#9F5B16,used,CEMBA171206_3C_AGCGATAGAACCAGGTAATGACGTCAGGACGT
2,CEMBA171206_3C,AGCGATAGAACCAGGTAGGATAACATAGAGGC,ASC,NonN,ASC,ASCC,#704E89,#A13F9E,#9C5B7E,used,CEMBA171206_3C_AGCGATAGAACCAGGTAGGATAACATAGAGGC
3,CEMBA171206_3C,AGCGATAGAACCAGGTATAGCCTTAGGCGAAG,L6.CT,Glutamate,CTGL,CTGL4,#8C4E3C,#A78533,#EEC4AD,used,CEMBA171206_3C_AGCGATAGAACCAGGTATAGCCTTAGGCGAAG
4,CEMBA171206_3C,AGCGATAGAACCAGGTATAGCCTTATAGAGGC,L5.IT.b,Glutamate,ITL6GL,ITL6GL1,#8C4E3C,#9E4954,#8C5E19,used,CEMBA171206_3C_AGCGATAGAACCAGGTATAGCCTTATAGAGGC


In [26]:
df_merge = pd.merge(df_old, df_new_update, left_on='cell', right_on='Li et al 2020 cell')
df_merge = df_merge.drop([
    'Li et al 2020 sample',
    'Li et al 2020 barcode', 
    'Li et al 2020 cluster', 
    'Li et al 2020 ClassColor',
    'Li et al 2020 MajorTypeColor', 
    'Li et al 2020 SubTypeColor',
    'Li et al 2020 note',
    'Li et al 2020 cell',
], axis=1)

print(df_merge.shape)
print(df_merge.columns)
df_merge.head()

(81196, 16)
Index(['cell', 'sample', 'barcode', 'Num fragments', 'Num uniquely mapped',
       'Num proper paired', 'Num unique fragments', 'Num chrM fragments',
       'cluster', 'tsne1', 'tsne2', 'umap-1', 'umap-2', 'Li et al 2020 Class',
       'Li et al 2020 MajorType', 'Li et al 2020 SubType'],
      dtype='object')


,cell,sample,barcode,Num fragments,Num uniquely mapped,Num proper paired,Num unique fragments,Num chrM fragments,cluster,tsne1,tsne2,umap-1,umap-2,Li et al 2020 Class,Li et al 2020 MajorType,Li et al 2020 SubType
0,CEMBA171206_3C_AGCGATAGAACCAGGTAAGAGATGTATAGCCT,CEMBA171206_3C,AGCGATAGAACCAGGTAAGAGATGTATAGCCT,138330,128263,128039,64868,0,L6.CT,7.605117,0.697672,-2.125204,-4.396653,NaN,NaN,NaN
1,CEMBA171206_3C_AGCGATAGAACCAGGTAATGACGTCAGGACGT,CEMBA171206_3C,AGCGATAGAACCAGGTAATGACGTCAGGACGT,159133,146663,146348,69167,0,L23.b,-9.492925,-6.200938,8.959074,3.006819,Glutamate,ITL23GL,ITL23GL1
2,CEMBA171206_3C_AGCGATAGAACCAGGTAGGATAACATAGAGGC,CEMBA171206_3C,AGCGATAGAACCAGGTAGGATAACATAGAGGC,19152,17586,17545,8348,0,ASC,14.227358,-11.023748,-6.880754,-2.903271,NonN,ASC,ASCC
3,CEMBA171206_3C_AGCGATAGAACCAGGTATAGCCTTAGGCGAAG,CEMBA171206_3C,AGCGATAGAACCAGGTATAGCCTTAGGCGAAG,14876,13837,13801,6583,0,L6.CT,5.144768,4.790674,-0.955707,-4.635163,Glutamate,CTGL,CTGL4
4,CEMBA171206_3C_AGCGATAGAACCAGGTATAGCCTTATAGAGGC,CEMBA171206_3C,AGCGATAGAACCAGGTATAGCCTTATAGAGGC,28532,26303,26202,12437,0,L5.IT.b,5.218895,-8.674734,0.796851,4.447933,Glutamate,ITL6GL,ITL6GL1


In [28]:
output = '/cndd2/fangming/projects/miniatlas/data/Table_S4g_open_chromatin_metadata.tsv.gz'
df_merge.to_csv(output, sep='\t', header=True, index=False, na_rep='NA')

### Check others 

In [30]:
print(tmp.shape)
tmp.head()

(408885, 14)


,sample,joint_embedding_x,joint_embedding_y,modality,modality_name,single_modality_cluster,single_modality_annot,joint_cluster_round1,joint_cluster_round2,joint_cluster_round3,joint_cluster_round4,joint_cluster_round1_annotation,joint_cluster_round2_annotation,joint_cluster_round3_annotation
0,snmcseq_gene_2C_M_0,0.471629,-3.134735,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Tenm2,L4-IT-Rorb_Rorb-Tenm2,1,1-1,1-1-1,1-1-1-1,L4/5 IT,L4/5 IT (1),L4/5 IT (1)
1,snmcseq_gene_2C_M_1011,-0.091199,-2.604921,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Tenm2,L4-IT-Rorb_Rorb-Tenm2,1,1-1,1-1-1,1-1-1-1,L4/5 IT,L4/5 IT (1),L4/5 IT (1)
2,snmcseq_gene_2C_M_1012,-0.037576,-8.106225,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,1,1-1,1-1-1,1-1-1-1,L4/5 IT,L4/5 IT (1),L4/5 IT (1)
3,snmcseq_gene_2C_M_1022,0.906631,-0.785923,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Cpne4,L4-IT-Rorb_Rorb-Cpne4,1,1-1,1-1-1,1-1-1-1,L4/5 IT,L4/5 IT (1),L4/5 IT (1)
4,snmcseq_gene_2C_M_1024,1.627603,-1.945875,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Cpne4,L4-IT-Rorb_Rorb-Cpne4,1,1-1,1-1-1,1-1-1-1,L4/5 IT,L4/5 IT (1),L4/5 IT (1)


In [32]:
tmp_atac = tmp[tmp['modality']=='snatac_gene']
print(tmp_atac.shape)

(54844, 14)


In [36]:
tmp_df_old = df_old.copy()
tmp_df_old['cell'] = 'snatac_gene_' + df_old['cell']
print(tmp_df_old.shape)

tmp_atac_merge = pd.merge(tmp_atac, tmp_df_old, left_on='sample', right_on='cell', how='left')
print(tmp_atac_merge.shape)
tmp_atac_merge.head()

(81196, 13)
(54844, 27)


,sample_x,joint_embedding_x,joint_embedding_y,modality,modality_name,single_modality_cluster,single_modality_annot,joint_cluster_round1,joint_cluster_round2,joint_cluster_round3,...,Num fragments,Num uniquely mapped,Num proper paired,Num unique fragments,Num chrM fragments,cluster,tsne1,tsne2,umap-1,umap-2
0,snatac_gene_CEMBA171206_3C_AGCGATAGAACCAGGTATT...,5.271253,-5.235372,snatac_gene,ATAC Seq,L4,L4,1,1-1,1-1-1,...,21895,20466,20417,10624,0,L4,-7.987973,-18.100446,5.171361,5.550792
1,snatac_gene_CEMBA171206_3C_AGCGATAGAAGAGGCAACT...,1.949641,-5.505484,snatac_gene,ATAC Seq,L4,L4,1,1-1,1-1-1,...,10346,9716,9699,6941,0,L4,-6.088012,-20.197620,5.419341,5.147774
2,snatac_gene_CEMBA171206_3C_AGCGATAGAAGAGGCAGTA...,2.465726,-4.176560,snatac_gene,ATAC Seq,L4,L4,1,1-1,1-1-1,...,10731,9733,9717,7109,0,L4,-12.362245,-22.860904,5.368538,4.390217
3,snatac_gene_CEMBA171206_3C_AGCGATAGAAGCAACGGGT...,3.266076,-7.583402,snatac_gene,ATAC Seq,L4,L4,1,1-1,1-1-1,...,8765,7976,7962,4144,0,L4,1.482309,-29.640123,4.245789,4.878218
4,snatac_gene_CEMBA171206_3C_AGCGATAGAAGCAACGTTA...,0.558153,-4.245329,snatac_gene,ATAC Seq,L4,L4,1,1-1,1-1-1,...,16384,14962,14928,7498,0,L4,-7.209250,-28.575377,5.186142,4.862025


In [39]:
np.sort(tmp_atac_merge['cluster'].unique())

array(['Chodl', 'L23.a', 'L23.b', 'L23.c', 'L4', 'L5.IT.a', 'L5.IT.b',
       'L5.PT', 'L6.CT', 'L6.IT', 'Lamp5_Arhgdib', 'Lamp5_Mettl21e',
       'Lamp5_Ndnf', 'Lamp5_Smad3', 'NP', 'Pv_Ntf3_Trim63', 'Pv_Tac1',
       'Pv_Vsig2', 'Sncg', 'Sst_Chrna2_Myh8', 'Sst_Man1a', 'Sst_Stk33',
       'Vip_Chat', 'Vip_Gcnt4', 'Vip_Hcls1', 'Vip_Lipg'], dtype=object)

In [40]:
np.sort(tmp_df_old['cluster'].unique())

array(['ASC', 'Chodl', 'Endo', 'L23.a', 'L23.b', 'L23.c', 'L4', 'L5.IT.a',
       'L5.IT.b', 'L5.PT', 'L6.CT', 'L6.IT', 'Lamp5_Arhgdib',
       'Lamp5_Mettl21e', 'Lamp5_Ndnf', 'Lamp5_Smad3', 'MGC', 'NP', 'OGC',
       'OPC', 'Other', 'Pv_Ntf3_Trim63', 'Pv_Tac1', 'Pv_Vsig2', 'Smc',
       'Sncg', 'Sst_Chrna2_Myh8', 'Sst_Man1a', 'Sst_Stk33', 'Vip_Chat',
       'Vip_Gcnt4', 'Vip_Hcls1', 'Vip_Lipg'], dtype=object)